# ETL for Google Maps data Files 

We will perform the extraction, transformation and data loading of the data included in the files provided in the Google Maps folder

In [1]:
import io
import os
import pandas as pd
import builtin.utils as ut


StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 3, Finished, Available)

___
## 1. Extract data

We going to extract the data from our Data Lake and in a first step, we going to concatenate all data in States folder and the merge with data in Metadata folder.

We create two variables (gm_path, metadata_sites) to store the paths of the Reviews and Metadata folders, data provided by Google Maps.

In [2]:
gm_path = '/lakehouse/default/Files/df_database/Review_estados_parquet'
metadata_sites = '/lakehouse/default/Files/df_database/Metadata_sitios_parquet/metadata-sitios.parquet'
keywords = ['museum', 'park', 'Hotel', 'Motel', 'Hostel', 'restaurant', 'Restaurant', 'forest', 'gallery', 'mall', 'pub', 'Zoo', 'roller coaster']

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 4, Finished, Available)

In [3]:
# Extracting Data
gm_df = ut.clean_states_and_concat(gm_path)
sites_df = ut.clean_sites(metadata_sites, keywords)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 5, Finished, Available)

## 1.1 Dropping unnecesary columns and duplicates

### States Dropping

In [4]:
ut.data_summ(gm_df)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 6, Finished, Available)

 Column                                     Data_type  No_miss_Qty  %Missing  Missing_Qty
user_id                             [<class 'float'>]     89946359      0.00            0
   name                               [<class 'str'>]     89946359      0.00            0
   time                               [<class 'int'>]     89946359      0.00            0
 rating                               [<class 'int'>]     89946359      0.00            0
   text           [<class 'str'>, <class 'NoneType'>]     50638615     43.70     39307744
   pics [<class 'NoneType'>, <class 'numpy.ndarray'>]      2495679     97.23     87450680
   resp          [<class 'NoneType'>, <class 'dict'>]     11029110     87.74     78917249
gmap_id                               [<class 'str'>]     89946359      0.00            0
   date                               [<class 'str'>]     89946359      0.00            0
  state                               [<class 'str'>]     89946359      0.00            0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,user_id,[<class 'float'>],89946359,0.00,0
1,name,[<class 'str'>],89946359,0.00,0
2,time,[<class 'int'>],89946359,0.00,0
3,rating,[<class 'int'>],89946359,0.00,0
4,text,"[<class 'str'>, <class 'NoneType'>]",50638615,43.70,39307744
5,pics,"[<class 'NoneType'>, <class 'numpy.ndarray'>]",2495679,97.23,87450680
6,resp,"[<class 'NoneType'>, <class 'dict'>]",11029110,87.74,78917249
7,gmap_id,[<class 'str'>],89946359,0.00,0
8,date,[<class 'str'>],89946359,0.00,0
9,state,[<class 'str'>],89946359,0.00,0


In [5]:
drop_col=['resp', 'pics', 'time']
gm_df.drop(columns=drop_col, inplace=True)
gm_df.drop_duplicates(inplace=True)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 7, Finished, Available)

In [6]:
unique_states = gm_df['state'].unique()
unique_states_list = list(unique_states)
print(unique_states_list)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 8, Finished, Available)

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District_of_Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New_Hampshire', 'New_Jersey', 'New_Mexico', 'New_York', 'North_Carolina', 'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode_Island', 'South_Carolina', 'South_Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming']


We going to select the most cultural and turistic states in the USA

In [7]:
# Lista de estados permitidos
allowed_states = ['California', 'Florida', 'New_York', 'Nevada', 'Indiana', 
                  'Massachusetts', 'Illinois', 'District_of_Columbia', 'Louisiana', 'Hawaii, ''South_Carolina']

# Filtrar el DataFrame
gm_df = gm_df[gm_df['state'].isin(allowed_states)]


StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 9, Finished, Available)

### Sites Dropping

In [8]:
ut.data_summ(sites_df)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 10, Finished, Available)

          Column                                     Data_type  No_miss_Qty  %Missing  Missing_Qty
            name                               [<class 'str'>]       277783      0.00            0
         address           [<class 'str'>, <class 'NoneType'>]       275798      0.71         1985
         gmap_id                               [<class 'str'>]       277783      0.00            0
     description           [<class 'NoneType'>, <class 'str'>]        80137     71.15       197646
        latitude                             [<class 'float'>]       277783      0.00            0
       longitude                             [<class 'float'>]       277783      0.00            0
        category                     [<class 'numpy.ndarray'>]       277783      0.00            0
      avg_rating                             [<class 'float'>]       277783      0.00            0
  num_of_reviews                               [<class 'int'>]       277783      0.00            0
          

,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,name,[<class 'str'>],277783,0.00,0
1,address,"[<class 'str'>, <class 'NoneType'>]",275798,0.71,1985
2,gmap_id,[<class 'str'>],277783,0.00,0
3,description,"[<class 'NoneType'>, <class 'str'>]",80137,71.15,197646
4,latitude,[<class 'float'>],277783,0.00,0
5,longitude,[<class 'float'>],277783,0.00,0
6,category,[<class 'numpy.ndarray'>],277783,0.00,0
7,avg_rating,[<class 'float'>],277783,0.00,0
8,num_of_reviews,[<class 'int'>],277783,0.00,0
9,price,"[<class 'NoneType'>, <class 'str'>]",101239,63.55,176544


In [9]:
    # Lista de columnas a eliminar
drop_col = ['price', 'url', 'hours', 'description', 'state', 'MISC', 'relative_results']
    
    # Eliminar columnas
sites_df.drop(columns=drop_col, inplace=True)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 11, Finished, Available)

In [10]:
gh_df = pd.merge(sites_df, gm_df, on = 'gmap_id', how = 'inner')

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 12, Finished, Available)

In [11]:
## Dataframe info
ut.data_summ(gh_df)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 13, Finished, Available)

        Column                           Data_type  No_miss_Qty  %Missing  Missing_Qty
        name_x                     [<class 'str'>]      4332565      0.00            0
       address [<class 'str'>, <class 'NoneType'>]      4330741      0.04         1824
       gmap_id                     [<class 'str'>]      4332565      0.00            0
      latitude                   [<class 'float'>]      4332565      0.00            0
     longitude                   [<class 'float'>]      4332565      0.00            0
      category           [<class 'numpy.ndarray'>]      4332565      0.00            0
    avg_rating                   [<class 'float'>]      4332565      0.00            0
num_of_reviews                     [<class 'int'>]      4332565      0.00            0
       user_id                   [<class 'float'>]      4332565      0.00            0
        name_y                     [<class 'str'>]      4332565      0.00            0
        rating                     [<class 

,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,name_x,[<class 'str'>],4332565,0.00,0
1,address,"[<class 'str'>, <class 'NoneType'>]",4330741,0.04,1824
2,gmap_id,[<class 'str'>],4332565,0.00,0
3,latitude,[<class 'float'>],4332565,0.00,0
4,longitude,[<class 'float'>],4332565,0.00,0
5,category,[<class 'numpy.ndarray'>],4332565,0.00,0
6,avg_rating,[<class 'float'>],4332565,0.00,0
7,num_of_reviews,[<class 'int'>],4332565,0.00,0
8,user_id,[<class 'float'>],4332565,0.00,0
9,name_y,[<class 'str'>],4332565,0.00,0


In [12]:
##Desanidado de datos que contiene la columna "category"
dataframe = gh_df
column = 'category'
cat = ut.explode_column(dataframe, column)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 14, Finished, Available)

In [13]:
##Reemplazar valores nulos
cat = ut.replace_all_nulls(cat)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 15, Finished, Available)

In [14]:
#Visualizar todas las categorias desanidadas
unique_categories = cat['category'].unique()
for category in unique_categories:
    print(category)


StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 16, Finished, Available)

Korean restaurant
Restaurant
Mailing service
Courier service
Fax service
Lamination service
Mailbox rental service
Notary public
Packaging supply store
Post office
Shipping service
Seafood restaurant
Mexican restaurant
RV park
Campground
Coffee shop
Bagel shop
Bakery
Breakfast restaurant
Donut shop
Fast food restaurant
Takeout Restaurant
Modern French restaurant
Pizza restaurant
Italian restaurant
Delivery Restaurant
Public parking space
Ecological park
Skateboard park
Hamburger restaurant
American restaurant
Barbecue restaurant
Sandwich shop
Shopping mall
Art gallery
Indian restaurant
Cafeteria
Vegan restaurant
Bistro
Kosher restaurant
Vegetarian restaurant
Cafe
Lunch restaurant
Shipping and mailing service
Business center
Freight forwarding service
Office supply store
Print shop
Shredding service
Gas station
ATM
Convenience store
Chinese restaurant
Asian fusion restaurant
Chinese noodle restaurant
Dumpling restaurant
Soup restaurant
Juice shop
Health food restaurant
Health food store

In [15]:
#Eliminar datos correspondientes a negocios que no son de nuestro interes a procesar

keywords = ['museum', 'park', 'Hotel', 'Motel', 'Hostel', 'restaurant', 'Restaurant', 'forest', 'gallery', 'mall', 'pub', 'Zoo', 'roller coaster']

# Convertir las palabras clave a minúsculas para que la búsqueda sea insensible a mayúsculas
keywords_lower = [keyword.lower() for keyword in keywords]

# Crear una máscara booleana para filtrar las filas
mask = cat['category'].str.lower().str.contains('|'.join(keywords_lower))

# Aplicar la máscara para conservar solo las filas que contienen las palabras clave
gh_df = cat[mask]


StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 17, Finished, Available)

In [16]:
##Aplicar filtro
gh_df = gh_df.reset_index(drop=True)
gh_df

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 18, Finished, Available)

,name_x,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,user_id,name_y,rating,text,date,state
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,1.089912e+20,Song Ro,5,Love there korean rice cake.,2021-01-06 05:12:07,California
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,1.112903e+20,Rafa Robles,5,Good very good,2021-02-09 05:47:28,California
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,1.126404e+20,David Han,4,They make Korean traditional food very properly.,2020-03-08 05:04:42,California
3,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,1.174403e+20,Anthony Kim,5,Short ribs are very delicious.,2019-03-07 05:56:56,California
4,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,1.005808e+20,Mario Marzouk,5,Great food and prices the portions are large,2017-05-16 05:01:41,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10298003,The Green Room Of Libertyville,"The Green Room Of Libertyville, 624 Milwaukee ...",0x880f96c84b0ef57b:0xe75b67b15a2cfdad,42.289450,-87.954386,Restaurant,4.6,228,1.174275e+20,Max Eggers,3,ND,2019-01-13 04:40:40,Illinois
10298004,The Green Room Of Libertyville,"The Green Room Of Libertyville, 624 Milwaukee ...",0x880f96c84b0ef57b:0xe75b67b15a2cfdad,42.289450,-87.954386,Restaurant,4.6,228,1.077471e+20,Brian Lawton,5,ND,2019-10-20 04:31:09,Illinois
10298005,The Green Room Of Libertyville,"The Green Room Of Libertyville, 624 Milwaukee ...",0x880f96c84b0ef57b:0xe75b67b15a2cfdad,42.289450,-87.954386,Restaurant,4.6,228,1.114333e+20,Christopher Cantagallo,4,ND,2017-10-18 13:47:06,Illinois
10298006,The Green Room Of Libertyville,"The Green Room Of Libertyville, 624 Milwaukee ...",0x880f96c84b0ef57b:0xe75b67b15a2cfdad,42.289450,-87.954386,Restaurant,4.6,228,1.103616e+20,Saira S,5,ND,2019-01-09 11:38:23,Illinois


In [17]:
##agrupar nuevamente las categorias y el resto de la información
gmap = ut.group_column(gh_df, 'category')
gmap.head(2)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 19, Finished, Available)

,name_x,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews,user_id,name_y,rating,text,date,state,category
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,4.4,18,1.089912e+20,Song Ro,5,Love there korean rice cake.,2021-01-06 05:12:07,California,[Korean restaurant]
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,4.4,18,1.112903e+20,Rafa Robles,5,Good very good,2021-02-09 05:47:28,California,[Korean restaurant]


In [18]:
df1 = gmap[['name_x', 'longitude', 'latitude']]
df1

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 20, Finished, Available)

,name_x,longitude,latitude
0,San Soo Dang,-118.292130,34.058092
1,San Soo Dang,-118.292130,34.058092
2,San Soo Dang,-118.292130,34.058092
3,San Soo Dang,-118.292130,34.058092
4,San Soo Dang,-118.292130,34.058092
...,...,...,...
10298003,The Green Room Of Libertyville,-87.954386,42.289450
10298004,The Green Room Of Libertyville,-87.954386,42.289450
10298005,The Green Room Of Libertyville,-87.954386,42.289450
10298006,The Green Room Of Libertyville,-87.954386,42.289450


In [19]:
df1 = df1.drop_duplicates().reset_index(drop=True)
df1

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 21, Finished, Available)

,name_x,longitude,latitude
0,San Soo Dang,-118.292130,34.058092
1,Vons Chicken,-118.010855,33.916402
2,Mail Station,-82.508387,27.389683
3,Long John Silver's,-87.296512,41.472375
4,Taqueria El Patron,-92.040551,30.260098
...,...,...,...
49932,Farmer Meat Market,-91.169125,30.493538
49933,China King Express,-78.988253,42.698185
49934,D Cuisine,-87.644375,41.932084
49935,House of Gourmet -- 食全食美,-78.827337,42.980251


In [20]:
#Agarra las cordenadas y a partir de la API se le asigna la ciudad donde está localizada

file_location = '/lakehouse/default/Files/df_database/city.parquet'
api_key = 'AkAEqJCFD-79UwAbaZa2rFXHVTUb1GEJb-3u5DkEIYV7c-uHbl50Ij7frtZdO3Y5'
df_city = ut.city_dataframe(df1, file_location, api_key)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 22, Finished, Available)

In [21]:
df_city.head(2)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 23, Finished, Available)

,name_x,longitude,latitude,City
0,San Soo Dang,-118.292130,34.058092,Los Angeles
1,Vons Chicken,-118.010855,33.916402,La Mirada


In [22]:
ut.data_summ(df_city)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 24, Finished, Available)

   Column                           Data_type  No_miss_Qty  %Missing  Missing_Qty
   name_x                     [<class 'str'>]        49937      0.00            0
longitude                   [<class 'float'>]        49937      0.00            0
 latitude                   [<class 'float'>]        49937      0.00            0
     City [<class 'str'>, <class 'NoneType'>]        45397      9.09         4540


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,name_x,[<class 'str'>],49937,0.00,0
1,longitude,[<class 'float'>],49937,0.00,0
2,latitude,[<class 'float'>],49937,0.00,0
3,City,"[<class 'str'>, <class 'NoneType'>]",45397,9.09,4540


In [23]:
df_city = df_city.dropna(subset=['City'])

# Restablecer los índices después de eliminar filas
df_city = df_city.reset_index(drop=True)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 25, Finished, Available)

In [24]:
df_maps = pd.merge(gmap, df_city, on=["name_x", "longitude", "latitude"], how = "inner")

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 26, Finished, Available)

In [25]:
df_maps["count_checkin"] = 1

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 27, Finished, Available)

In [26]:
df_maps.head(2)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 28, Finished, Available)

,name_x,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews,user_id,name_y,rating,text,date,state,category,City,count_checkin
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,4.4,18,1.089912e+20,Song Ro,5,Love there korean rice cake.,2021-01-06 05:12:07,California,[Korean restaurant],Los Angeles,1
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,4.4,18,1.112903e+20,Rafa Robles,5,Good very good,2021-02-09 05:47:28,California,[Korean restaurant],Los Angeles,1


In [27]:
df_maps = df_maps[["name_x", "address", "City", "state", "latitude", "longitude", "avg_rating", "num_of_reviews", "date", "count_checkin", "user_id", "rating", "text", "name_y", "category"]]

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 29, Finished, Available)

In [28]:
df_maps = df_maps.rename(columns={
    "name_x": 'Business_Name',
    "address": 'Address',
    "City": 'City',
    "state": 'State',
    "latitude": 'Latitude',
    "longitude": 'Longitude',
    "avg_rating": 'Ranking',
    "num_of_reviews": 'Review_Count',
    "date" : "Date",
    "count_checkin": 'Checkin_Count',
    "user_id": 'User_Id',
    "rating": 'Stars',
    "text": 'Text',
    "name_y": 'User_Name',
    "category": 'Category'
})

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 30, Finished, Available)

In [29]:
df_maps.head(2)

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 31, Finished, Available)

,Business_Name,Address,City,State,Latitude,Longitude,Ranking,Review_Count,Date,Checkin_Count,User_Id,Stars,Text,User_Name,Category
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Los Angeles,California,34.058092,-118.29213,4.4,18,2021-01-06 05:12:07,1,1.089912e+20,5,Love there korean rice cake.,Song Ro,[Korean restaurant]
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Los Angeles,California,34.058092,-118.29213,4.4,18,2021-02-09 05:47:28,1,1.112903e+20,5,Good very good,Rafa Robles,[Korean restaurant]


In [30]:
df_maps.to_parquet('/lakehouse/default/Files/df_database/files_to_EDA/Maps_to_EDA.parquet')

StatementMeta(, 905966e0-c7c4-4400-aa7f-c5319188fea7, 32, Finished, Available)